## Agent ReAct
ReAct to wzorzec łączący Reasoning + Acting: model krokowo rozumuje (planuje), po czym wykonuje akcje narzędziowe (zapytania do API, wyszukiwanie, obliczenia), a otrzymane obserwacje włączane są do kolejnych kroków rozumowania.

### Instalacja bibliotek

In [1]:
!pip install -q langchain langchain-openai python-dotenv

### Przygotowanie modelu

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Model gotowy.")


Model gotowy.


### W jaki sposób model korzysta z narzędzi?

In [3]:
import re

# narzędzie - tool
def greet_user(name: str) -> str:
    """Funkcja witająca użytkownika po imieniu."""
    return f"Witaj, {name}! Miło Cię poznać!"

prompt = """
Odpowiedz na pytanie użytkownika.
Jeśli użytkownik przedstawi się, użyj formatu: GREET[imię] i więcej nie używaj imienia użytkownika.

Pytanie: Cześć, nazywam się Michał i chciałbym się dowiedzieć co było pierwsze - jajko czy gniazdo?
"""

response = llm.invoke(prompt)
response_text = response.content

pattern = r'GREET\[(\w+)\]'
match = re.search(pattern, response_text)

# Jeśli znaleziono wzorzec, wywołaj funkcję-narzędzie
if match:
    name = match.group(1)
    print("Wykryto wzorzec GREET, wywołuję funkcję-narzędzie...")
    result = greet_user(name)
    print(f"Wynik funkcji: {result}")
    response_text = response_text.replace(match.group(0), result)
else:
    print("Nie znaleziono wzorca GREET w odpowiedzi.")

print("Odpowiedź LLM:")
print(response_text)

Wykryto wzorzec GREET, wywołuję funkcję-narzędzie...
Wynik funkcji: Witaj, Michał! Miło Cię poznać!
Odpowiedź LLM:
Witaj, Michał! Miło Cię poznać! To ciekawe pytanie! W kontekście ewolucji, jajko pojawiło się przed gniazdem. Jajka były składane przez zwierzęta na długo przed tym, jak ptaki zaczęły budować gniazda. Wiele prehistorycznych stworzeń, takich jak gady, składało jajka, zanim pojawiły się ptaki i ich gniazda.


### Definiowanie narzędzi

In [4]:
from langchain_classic.tools import tool

@tool
def add_numbers(**kwargs:int) -> int:
    """Add numbers together."""
    return sum(kwargs['kwargs'].values())

@tool
def company_info(name: str) -> str:
    "return info about company"
    data = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT. Istnieje od 10 lat.",
        "LangChain": "Framework do budowania aplikacji AI z LLM. Istnieje od 4 lat.",
        "Tesla": "Producent samochodów elektrycznych. Istnieje 23 lata."
    }
    return data.get(name, "Nie mam informacji o tej firmie.")

tools = [add_numbers, company_info]

### Agent ReAct

In [5]:
from langchain_classic.agents import initialize_agent, AgentType

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=6
)

agent.run("Powiedz mi, czym zajmują się firmy LangChain, Tesla oraz OpenAI. Następnie podaj sumę liczby lat na rynku tych 3 firm.")

/tmp/ipykernel_30711/3825512292.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(
/tmp/ipykernel_30711/3825512292.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  agent.run("Powiedz mi, czym zajmują się firmy LangChain, Tesla oraz OpenAI. Następnie podaj sumę liczby lat na rynk



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "LangChain"
  }
}
```
Observation: Framework do budowania aplikacji AI z LLM. Istnieje od 4 lat.
Thought:Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "Tesla"
  }
}
```
Observation: Producent samochodów elektrycznych. Istnieje 23 lata.
Thought:Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "OpenAI"
  }
}
```

Observation: Firma zajmująca się sztuczną inteligencją, twórca ChatGPT. Istnieje od 10 lat.
Thought:Action:
```
{
  "action": "add_numbers",
  "action_input": {
    "kwargs": {
      "LangChain": 4,
      "Tesla": 23,
      "OpenAI": 10
    }
  }
}
```
Observation: 37
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "LangChain zajmuje się tworzeniem frameworków do budowania aplikacji AI z LLM i istnieje od 4 lat. Tesla to producent samochodów elektrycznych, który istnieje od 23 lat. OpenAI to fir

'LangChain zajmuje się tworzeniem frameworków do budowania aplikacji AI z LLM i istnieje od 4 lat. Tesla to producent samochodów elektrycznych, który istnieje od 23 lat. OpenAI to firma zajmująca się sztuczną inteligencją, twórca ChatGPT, istniejąca od 10 lat. Suma liczby lat na rynku tych trzech firm wynosi 37 lat.'